In [2]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import pandas as pd
import numpy as np
import re
import string

In [3]:
pd.set_option("display.max_rows",None)

In [3]:
chrome_path = './chromedriver/chromedriver'
driver = webdriver.Chrome(chrome_path)

In [4]:
#Countries
Singapore = 'https://myjobstreet.jobstreet.com.sg/home/login.php?site=SG'
Malaysia = 'https://myjobstreet.jobstreet.com.my/home/login.php'
Philippines = 'https://www.jobstreet.com.ph/'
Indonesia = 'https://www.jobstreet.co.id/'

In [5]:
#Code for scraping jobstreet and making dataframe
def scrape_jobstreet(job_titles, pages, country):
    jobs_df = []
    companies_df = []
    locations_df = []
    salaries_df = []
    industries_df = []
    experiences_df = []
    company_sizes_df = []
    descriptions_df = []
    linkss_df = []
    
    def jjobs(info):
        jobs = []
        companies = []
        locations = []
        linkss = []
        for inf in info:
            job = inf.findAll('h2', {'itemprop':'title'})
            for j in job:
                jobs.append(j.text)
            company = inf.findAll('span', {'itemprop':'name'})
            for com in company:
                companies.append(com.text)
            location = inf.findAll('span', {'itemprop':'jobLocation'})
            for loc in location:
                locations.append(loc.text)
            links = inf.findAll('a', {'class', 'position-title-link'})
            for link in links:
                linkss.append(link.get('href'))
        return jobs, companies, locations, linkss
    
    def get_info(linkss):
        salaries = []
        industries = []
        experiences = []
        company_sizes =[]
        descriptions = []
        for link in linkss:
            driver.get(link)
            source = driver.page_source
            soup = BeautifulSoup(source, "html.parser")
            try:
                sal = soup.findAll('span', {'id':'salary_range'})[0].text.strip()
                salaries.append(sal)
            except:
                salaries.append(np.nan)
            try:
                ind = soup.findAll('p', {'id':'company_industry'})[0].text
                industries.append(ind)
            except:
                industries.append(np.nan)
            try:
                exp = soup.findAll('span', {'itemprop':'experienceRequirements'})[0].text.strip()
                experiences.append(exp)
            except:
                experiences.append(np.nan)
            try:
                com = soup.findAll('p', {'id':'company_size'})[0].text.strip()
                company_sizes.append(com)
            except:
                company_sizes.append(com)
            try:
                des = soup.findAll('div', attrs = {'id':'job_description'})[0].text
                descriptions.append(des)
            except:
                descriptions.append(np.nan)
            time.sleep(4)
        return salaries, industries, experiences, company_sizes, descriptions 
    
    #user logins to profile first
    #driver.get(country)
    for j in job_titles:
        for p in range(1, pages+1):
            if country == Singapore:
                url = 'https://www.jobstreet.com.sg/en/job-search/job-vacancy.php?key=' + j.replace(' ', '+') + '&area=1&option=1&job-source=1%2C64&classified=1&job-posted=0&sort=2&order=0&pg=' + str(p) + '&src=16&ojs=2'
            elif country == Malaysia:
                url = 'https://www.jobstreet.com.my/en/job-search/job-vacancy.php?key=' + j.replace(' ', '+') + '&area=1&option=1&job-source=1%2C64&classified=1&job-posted=0&sort=2&order=0&pg=' + str(p) + '&src=16&ojs=2'
            elif country == Philippines:
                url = 'https://www.jobstreet.com.ph/en/job-search/job-vacancy.php?key=' + j.replace(' ', '+') + '&area=1&option=1&job-source=1%2C64&classified=1&job-posted=0&sort=2&order=0&pg=' + str(p) + '&src=16&ojs=2'
            elif country == Indonesia:
                url = 'https://www.jobstreet.co.id/en/job-search/job-vacancy.php?key=' + j.replace(' ', '+') + '&area=1&option=1&job-source=1%2C64&classified=1&job-posted=0&sort=2&order=0&pg=' + str(p) + '&src=16&ojs=2'
                driver.get(url)
                source = driver.page_source
                soup = BeautifulSoup(source, "html.parser")
                info = soup.findAll('div', {'class':'panel-body'})
            
            a,b,c,d = jjobs(info)
            
            jobs_df.extend(a)
            companies_df.extend(b)
            locations_df.extend(c)
            linkss_df.extend(d)
            
            e,f,g,h,i = get_info(d)
            
            salaries_df.extend(e)
            industries_df.extend(f)
            experiences_df.extend(g)
            company_sizes_df.extend(h)
            descriptions_df.extend(i)
            
            time.sleep(4)
            
        df = pd.DataFrame()
        df['job_title'] = jobs_df
        df['company'] = companies_df
        df['location'] = locations_df
        df['salary'] = salaries_df
        df['industry'] = industries_df
        df['experience'] = experiences_df
        df['company_size'] = company_sizes_df
        df['job_description'] = descriptions_df
    return df

In [8]:
#Run driver first
driver.get(Singapore)

In [175]:
#Sample dataframe
dd = scrape_jobstreet(['data analyst','data scientist'], 2, Singapore)

In [ ]:
#Expected salary is SGD 3000

In [176]:
dd

,job_title,company,location,salary,industry,experience,company_size,job_description
0,Data Analyst,VENIAM ASIA PTE. LTD.,Singapore,"SGD 3,000 - SGD 6,000",Telecommunication,Min 1 year (Junior Executive),1 - 50 Employees,\nVeniam’s vehicular networks collect all sor...
1,Data Analyst,Brightoil Petroleum (S’pore) Pte. Ltd.,West,Around expected salary,Oil/Gas/Petroleum,Min 3 years (Junior Executive),51 - 200 Employees,Job Responsibilities• Performing data extract...
2,Data Analyst,Hotelbeds Pte. Ltd.,Novena (Central),"SGD 3,500 - SGD 4,900",Travel/Tourism,Min 3 years (Entry Level),51 - 200 Employees,\nAt A Glance\n\nThis role’s main purpose is ...
3,Data Analyst (1 Year Contract),DN & ASSOCIATES EXECUTIVE SEARCH PTE. LTD.,Central,Above expected salary,Human Resources Management/Consulting,Min 2 years (Junior Executive),1 - 50 Employees,RESPONSIBILITIES:Track performance of lead ge...
4,Data Analyst - (004CZ),National University of Singapore,Singapore - Singapore,NaN,Education,Junior Executive,More than 5000 Employees,"To work in a team that source, collect, clean..."
5,"Executive / Senior Executive, CapitaStar (Data...",CapitaLand Limited,Central,Above expected salary,Property/Real Estate,Min 2 years (Junior Executive),1001 - 2000 Employees,ResponsibilitiesData Analyst on our team cont...
6,Reporting / Data Analyst [W],ST Electronics (e-Services) Pte Ltd,Central,"SGD 2,500 - SGD 2,800",Human Resources Management/Consulting,Min 3 years (Junior Executive),201 - 500 Employees,Roles and ResponsibilitiesPerform data update...
7,Data Analyst (6 months / Central / Up to $3.8K),PRIMESTAFF MANAGEMENT SERVICES PTE LTD,Central - Raffles Place,"SGD 3,000 - SGD 3,500",Human Resources Management/Consulting,Min 2 years (Senior Executive),1 - 50 Employees,Data AnalystMNC / 5 Day Work Week6 Months Con...
8,Data Analyst (Python / SAS / SQL)(BANK / $3.5K...,Achieve Career Consultant Pte Ltd,Singapore,NaN,Human Resources Management/Consulting,Min 2 years (Junior Executive),51 - 200 Employees,"S$3,500 to S$5,000 or Higher!Good Career Prog..."
9,Data Quality Analyst - Private Banking,CHANDLER MACLEOD GROUP PTE LTD,Central,"SGD 2,400 - SGD 2,600",Human Resources Management/Consulting,Min 1 year (Junior Executive),1 - 50 Employees,"Immediate - Jun 2018CBD, $2600 MonthlyPaid An..."


### Data Cleaning and Feature Encoding

In [5]:
sg = pd.read_csv('./sg_jobst')
mal = pd.read_csv('./mal_jobst')
phil = pd.read_csv('./phil_jobst')
ind = pd.read_csv('./ind_jobst')

In [6]:
#Change location before concatenating all the dataframes together
sg['location'] = sg['location'].apply(lambda x: 'Singapore')
mal['location'] = mal['location'].apply(lambda x: 'Malaysia')
phil['location'] = phil['location'].apply(lambda x: 'Philippines')
ind['location'] = ind['location'].apply(lambda x: 'Indonesia')

In [7]:
#concat all these dataframes together
jobst = pd.concat([sg,mal,phil,ind])

In [8]:
jobst.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1601 entries, 0 to 232
Data columns (total 9 columns):
Index              1601 non-null int64
job_title          1601 non-null object
company            1601 non-null object
location           1601 non-null object
salary             930 non-null object
industry           1572 non-null object
experience         1590 non-null object
company_size       1601 non-null object
job_description    1596 non-null object
dtypes: int64(1), object(8)
memory usage: 125.1+ KB


In [9]:
jobst.drop('Index', inplace=True)

In [10]:
jobst['Index'] = np.arange(jobst.shape[0])

In [11]:
jobst.set_index('Index', inplace=True)

In [12]:
jobst1 = jobst.dropna()

In [13]:
#jobst1.info()

In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
# drop duplicates
jobst1.drop_duplicates(inplace=True)

In [16]:
#jobst1.info()

In [17]:
# Need clean and encode each column

In [18]:
jobst1['job_title'] = jobst1['job_title'].apply(lambda x: 'Business Analyst' if 'business' in x.lower() else 'Data Scientist' if 'scientist' in x.lower() else 'Data Engineer' if 'engineer' in x.lower() else 'Data Analyst' if 'data' in x.lower() else np.nan)
                                                

In [19]:
jobst1['job_title'].value_counts(dropna=False)

Business Analyst    374
Data Analyst        176
NaN                 133
Data Scientist       68
Data Engineer        67
Name: job_title, dtype: int64

In [20]:
jobst1.dropna(inplace=True)

In [21]:
jobst1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 685 entries, 0 to 1586
Data columns (total 8 columns):
job_title          685 non-null object
company            685 non-null object
location           685 non-null object
salary             685 non-null object
industry           685 non-null object
experience         685 non-null object
company_size       685 non-null object
job_description    685 non-null object
dtypes: object(8)
memory usage: 48.2+ KB


In [22]:
jobst1.head(2)

,job_title,company,location,salary,industry,experience,company_size,job_description
Index,,,,,,,,
0,Data Analyst,Achieve Career Consultant Pte Ltd,Singapore,"SGD 3,500 - SGD 5,000",Human Resources Management/Consulting,Min 2 years (Junior Executive),51 - 200 Employees,"S$3,500 to S$5,000 or Higher!Good Career Prog..."
1,Data Analyst,CHANDLER MACLEOD GROUP PTE LTD,Singapore,"SGD 2,400 - SGD 2,600",Human Resources Management/Consulting,Min 1 year (Junior Executive),1 - 50 Employees,"Immediate - Jun 2018CBD, $2600 MonthlyPaid An..."


In [23]:
# Cleaning the salary column and doing currency conversion
jobst1['salary'] = jobst1['salary'].apply(lambda x: '>3000' if x=='Above expected salary' else '<=3000' if x=='Around expected salary' else '<=3000' if x=='Below expected salary' else x)

In [24]:
# Exchange rate
# 1 SGD == 3.11 MYR
# 1 SGD == 3191.76 IDR
# 1 SGD == 38.08 PHP
# 1 SGD == 0.74 USD

In [25]:
# function to convert the currency of the different countries and get the mean of the currencies
def convert_currency(data):
    if data=='>3000' or data=='<=3000':
        return data
    data = data.replace(',', '')
    data = data.replace('-','')
    if 'USD' in data:
        data = data.replace('USD','')
        data = data.split()
        mean = (float(data[0]) + float(data[1]))/2
        con = mean/0.74
        return con
    elif 'SGD' in data:
        data = data.replace('SGD','')
        data = data.split()
        mean = (float(data[0]) + float(data[1]))/2
        return mean
    elif 'MYR' in data:
        data = data.replace('MYR','')
        data = data.split()
        mean = (float(data[0]) + float(data[1]))/2
        con = mean/3.11
        return con
    elif 'IDR' in data:
        data = data.replace('IDR','')
        data = data.split()
        mean = (float(data[0]) + float(data[1]))/2
        con = mean/3191.76
        return con
    elif 'PHP' in data:
        data = data.replace('PHP','')
        data = data.split()
        mean = (float(data[0]) + float(data[1]))/2
        con = mean/38.08
        return con
    else:
        return np.nan

In [26]:
jobst1['salary'] = jobst1['salary'].apply(convert_currency)

In [27]:
# Convert the salary column to types--> greater than 3000 and less than 3000 to 1 and 0 respectively

In [28]:
def encode_salary(data):
    if data == '>3000':
        return 1
    elif data == '<=3000':
        return 0
    if type(data) == float:
        if data <= 3000:
            return 0
        elif data > 3000:
            return 1 

In [29]:
jobst1['salary'] = jobst1['salary'].apply(encode_salary)

In [30]:
# Will do pd.get_dummies() for industries and drop non-significant ones using filtering

In [31]:
# Encoding the experience column

--> Unique Titles
'Junior Executive', 'Manager', 'Senior Executive', 'Senior Manager', 'Non-Executive', 'Entry Level', 'Experienced Employee', 'Supervisor', 'Less than 1 year experience', 'Staff', 'Supervisor'

In [32]:
def code_experience(data):
    if 'Junior Executive' in data:
        return 'Junior_Executive'
    elif 'Manager' in data:
        return 'Manager'
    elif 'Senior Executive' in data:
        return 'Senior_Executive'
    elif 'Senior Manager' in data:
        return 'Senior_Manager'
    elif 'Non-Executive' in data:
        return 'Non_Executive'
    elif 'Entry Level' in data:
        return 'Entry_Level'
    elif 'Experienced Employee' in data:
        return 'Experienced_Employee'
    elif 'Supervisor' in data:
        return 'Supervisor'
    elif 'Less than 1 year experience' in data:
        return 'Less_than_1_year'
    elif 'Staff' in data:
        return 'Staff'
    elif 'Supervisor' in data:
        return 'Supervisor'

In [33]:
jobst1['experience'] = jobst1['experience'].apply(code_experience)

In [34]:
jobst1['company_size'].unique()

array(['51 - 200 Employees', '1 - 50 Employees', '201 - 500 Employees',
       'More than 5000 Employees', '501 - 1000 Employees',
       '1001 - 2000 Employees', '2001 - 5000 Employees'], dtype=object)

In [35]:
# Rank Hot encode the company size
def encode_company_size(data):
    if data == '1 - 50 Employees':
        return 0
    elif data == '51 - 200 Employees': 
        return 1
    elif data == '201 - 500 Employees':
        return 2
    elif data == '501 - 1000 Employees':
        return 3
    elif data == '1001 - 2000 Employees':
        return 4
    elif data == '2001 - 5000 Employees':
        return 5
    elif data == 'More than 5000 Employees':
        return 6

In [36]:
jobst1['company_size'] = jobst1['company_size'].apply(encode_company_size)

In [37]:
jobst1['company_size'].unique()

array([1, 0, 2, 6, 3, 4, 5])

In [38]:
# Since there are 410 companies, converting column to dummies will not give much information
# So drop the column
jobst2 = jobst1.drop('company', axis=1)

In [39]:
# Clean the 'job_description' column--> remove newline characters and punctuation marks
def clean_job(data):
    data = data.lower()
    punc = lambda x:x not in string.punctuation
    data = filter(punc, data)
    data = data.replace('\n','')
    return data

In [40]:
jobst2['job_description'] = jobst2['job_description'].apply(clean_job)

In [41]:
# Get the count of words in 'job_description' column and make new features corresponding to data keywords

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer(encoding='utf-8', stop_words='english')
cvec.fit(jobst2['job_description'])
job_words = pd.DataFrame(cvec.transform(jobst2['job_description']).todense(), columns=cvec.get_feature_names())

In [43]:
# Data related keywords
keywords = ['sql', 'sas', 'python', 'r', 'machine', 'mining', 'data', 'analysis', 'predict', 'ai', 'programming', 
            'tableau', 'algorithm', 'modeling', 'processing', 'visualisation', 'analytics', 'processing', 'debug',
           'database', 'statistics', 'statistical', 'predictive', 'coding', 'sap', 'business', 'aws', 'hadoop',
           'spark', 'wrangling', 'munging', 'hdfs', 'etl', 'api', 'spss', 'hive', 'pig', 'streaming', 'mapreduce',
           'weka', 'matlab', 'mysql', 'ms', 'server', 'oracle', 'db2', 'nosql', 'hbase', 'hana', 'mongodb', 'couchdb',
           'vertica', 'greenplum','pentaho', 'teradata', 'postresql']

In [44]:
count = 0
keys = []
for i in keywords:
    count += 1
    if i in job_words.columns:
        jobst2[i] = job_words[i]
        keys.append(i)
print count

56


In [45]:
#keys

In [46]:
jobst2.head(2)

,job_title,location,salary,industry,experience,company_size,job_description,sql,sas,python,...,oracle,db2,nosql,hbase,hana,mongodb,couchdb,vertica,pentaho,teradata
Index,,,,,,,,,,,,,,,,,,,,,
0,Data Analyst,Singapore,1,Human Resources Management/Consulting,Junior_Executive,1,s3500 to s5000 or highergood career progressi...,1.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Data Analyst,Singapore,0,Human Resources Management/Consulting,Junior_Executive,0,immediate jun 2018cbd 2600 monthlypaid annua...,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
# Condense all of these keywords into one representative column
summ = jobst2.loc[:,keys].sum(axis=1)

In [48]:
# Drop the rerepresented columns
jobst2.drop(keys, axis=1, inplace=True)
jobst2.drop('job_description', axis=1, inplace=True)

In [49]:
jobst2['skills'] = summ

In [50]:
jobst2.head()

,job_title,location,salary,industry,experience,company_size,skills
Index,,,,,,,
0,Data Analyst,Singapore,1,Human Resources Management/Consulting,Junior_Executive,1,17.0
1,Data Analyst,Singapore,0,Human Resources Management/Consulting,Junior_Executive,0,6.0
3,Data Analyst,Singapore,0,Human Resources Management/Consulting,Junior_Executive,2,2.0
4,Data Analyst,Singapore,0,Human Resources Management/Consulting,Junior_Executive,1,13.0
5,Data Analyst,Singapore,1,Consulting (Business & Management),Junior_Executive,0,26.0


In [51]:
# Cleaned dataframe to use for downstream processes in Qn1 and Qn2
jobst3 = jobst2.copy()

# Qn1

In [52]:
# Factors that impact salary

In [53]:
# Filtering variables

In [54]:
target = 'salary'
predictors = [i for i in jobst3.columns if i != target]

In [55]:
predictors

['job_title', 'location', 'industry', 'experience', 'company_size', 'skills']

In [56]:
#numerical predictors
num_predictors = ['skills']
cat_predictors = [i for i in predictors if i != 'skills']

In [57]:
#Filtering numerical variables
from sklearn.feature_selection import f_classif, chi2

num_sig = []
f,p = f_classif(jobst3[['skills']], jobst3[target])
if p <= 0.05:
    num_sig.append(i)
print num_sig

['skills']


In [58]:
# The numerical feature 'skills' is significantly related to the target variable salary

In [59]:
cat_predictors

['job_title', 'location', 'industry', 'experience', 'company_size']

In [60]:
# Get dataframe of categorical variables
jobst3_cat = jobst3.loc[:,cat_predictors]

In [61]:
jobst3_cat.head(2)

,job_title,location,industry,experience,company_size
Index,,,,,
0,Data Analyst,Singapore,Human Resources Management/Consulting,Junior_Executive,1
1,Data Analyst,Singapore,Human Resources Management/Consulting,Junior_Executive,0


In [62]:
# Filtering the categorical variables
jobst3_cat = pd.get_dummies(jobst3_cat, drop_first=True)

In [63]:
#Convert the columns to type 'int64'
for i in jobst3_cat.columns:
    jobst3_cat[i] = jobst3_cat[i].astype('int64')

In [64]:
cat_sig = []
for i in jobst3_cat.columns:
    y = np.array(jobst3[target])
    ch, p = chi2(jobst3_cat[[i]],y)
    if p<=0.05:
        cat_sig.append(i)
print cat_sig        

['company_size', 'job_title_Data Analyst', 'job_title_Data Scientist', 'location_Malaysia', 'location_Philippines', 'location_Singapore', 'industry_Call Center/IT-Enabled Services/BPO', 'industry_Computer/Information Technology (Software)', 'industry_Construction/Building/Engineering', 'industry_Human Resources Management/Consulting', 'industry_R&D', 'industry_Retail/Merchandise', 'industry_Telecommunication', 'experience_Experienced_Employee', 'experience_Junior_Executive', 'experience_Less_than_1_year', 'experience_Manager', 'experience_Non_Executive', 'experience_Senior_Executive', 'experience_Staff', 'experience_Supervisor']


In [65]:
#Building the filtered dataframe containing the significant variables

In [66]:
cat_sig_df = jobst3_cat.loc[:, cat_sig]
num_sig_df = jobst3.loc[:, num_sig]
jobst_sig = cat_sig_df.merge(num_sig_df, left_index=True, right_index=True, how='outer')
jobst_sig['salary'] = jobst3['salary']

In [67]:
jobst_sig.head(2)

,company_size,job_title_Data Analyst,job_title_Data Scientist,location_Malaysia,location_Philippines,location_Singapore,industry_Call Center/IT-Enabled Services/BPO,industry_Computer/Information Technology (Software),industry_Construction/Building/Engineering,industry_Human Resources Management/Consulting,...,experience_Experienced_Employee,experience_Junior_Executive,experience_Less_than_1_year,experience_Manager,experience_Non_Executive,experience_Senior_Executive,experience_Staff,experience_Supervisor,skills,salary
Index,,,,,,,,,,,,,,,,,,,,,
0,1,1,0,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,17.0,1
1,0,1,0,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,6.0,0


### Train-test split plus SMOTEENN

In [68]:
# Looking at the distribution of target variable
dist = jobst_sig['salary'].value_counts()
dist

0    467
1    218
Name: salary, dtype: int64

In [69]:
# Baseline accuracy
ratio_0 = float(dist[0])/(dist[0] + dist[1])
ratio_1 = float(dist[1])/(dist[0] + dist[1])
print 'class 0 ratio', ratio_0
print 'class 1 ratio', ratio_1
print 'Baseline accuracy', ratio_0

class 0 ratio 0.681751824818
class 1 ratio 0.318248175182
Baseline accuracy 0.681751824818


In [70]:
# Since unequal class distribution, need oversample minority class
# Do oversampling after train-test-split
from sklearn.model_selection import train_test_split

X = jobst_sig.loc[:, [i for i in jobst_sig.columns if i != 'salary']]
y = jobst_sig.loc[:,'salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, stratify=y, random_state=12)

In [71]:
#count of each class in y_train
num_zeros = (y_train == 0).sum()
num_ones = (y_train == 1).sum()
print num_zeros
print num_ones

327
152


In [72]:
from imblearn.combine import SMOTEENN
sm = SMOTEENN(ratio={0:400,1:400}, random_state=12, n_jobs=-1)
X_sm, y_sm = sm.fit_sample(X_train, y_train)

/Users/dakshi/anaconda2/lib/python2.7/site-packages/imblearn/utils/deprecation.py:50: DeprecationWarning: 'k' is deprecated from 0.2 and will be removed in 0.4. Use 'k_neighbors' instead.
  category=DeprecationWarning)
/Users/dakshi/anaconda2/lib/python2.7/site-packages/imblearn/utils/deprecation.py:50: DeprecationWarning: 'm' is deprecated from 0.2 and will be removed in 0.4. Use 'm_neighbors' instead.
  category=DeprecationWarning)


In [73]:
#count of each class
num_zeros = (y_sm == 0).sum()
num_ones = (y_sm == 1).sum()
print num_zeros
print num_ones

333
348


### Modeling

In [74]:
# Fit first model with LogisticRegressionCV, second model with RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier

log = LogisticRegressionCV(penalty='l1', solver='liblinear', cv=3, n_jobs=-1)
log.fit(X_sm, y_sm)
y_pred = log.predict(X_test)
print 'score:', log.score(X_sm, y_sm)
print 'score:', log.score(X_test, y_test)

score: 0.992657856094
score: 0.902912621359


In [75]:
log_coefs = pd.DataFrame({'variable':X_train.columns, 'coef':log.coef_[0],'abs_coef':np.abs(log.coef_[0])})

log_coefs.sort_values('abs_coef', inplace=True, ascending=False)

# Factors that impact salary in order of decreasing impact
log_coefs

,abs_coef,coef,variable
16,17.791047,17.791047,experience_Manager
19,15.820999,15.820999,experience_Staff
5,13.605292,13.605292,location_Singapore
18,11.761691,11.761691,experience_Senior_Executive
4,7.221432,-7.221432,location_Philippines
14,7.105564,7.105564,experience_Junior_Executive
17,5.679552,5.679552,experience_Non_Executive
11,5.308347,-5.308347,industry_Retail/Merchandise
9,4.527844,4.527844,industry_Human Resources Management/Consulting
1,3.421743,-3.421743,job_title_Data Analyst


In [76]:
from sklearn.metrics import classification_report
print classification_report(y_test, y_pred)

             precision    recall  f1-score   support

          0       0.96      0.89      0.93       140
          1       0.80      0.92      0.86        66

avg / total       0.91      0.90      0.90       206



In [77]:
# RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100, random_state=12)
forest.fit(X_sm, y_sm)
y_pred = log.predict(X_test)
print 'score:', log.score(X_sm, y_sm)
print 'score:', log.score(X_test, y_test)

score: 0.992657856094
score: 0.902912621359


In [78]:
forest_features = pd.DataFrame({'variable':X_train.columns, 'coef':forest.feature_importances_})

forest_features.sort_values('coef', inplace=True, ascending=False)

# Factors that impact salary in order of decreasing impact
forest_features

,coef,variable
5,0.301678,location_Singapore
21,0.156442,skills
4,0.103350,location_Philippines
18,0.099739,experience_Senior_Executive
9,0.091373,industry_Human Resources Management/Consulting
13,0.056110,experience_Experienced_Employee
0,0.045105,company_size
3,0.028450,location_Malaysia
16,0.025136,experience_Manager
6,0.017533,industry_Call Center/IT-Enabled Services/BPO


In [79]:
print classification_report(y_test, y_pred)

             precision    recall  f1-score   support

          0       0.96      0.89      0.93       140
          1       0.80      0.92      0.86        66

avg / total       0.91      0.90      0.90       206



**Summary:**
> Looking at the coefficients of the features from the LogisticRegression and RandomForest, the top 5 common features that impact salary are 'location_Singapore', 'experience_Senior_Executive', 'location_Philippines', 'experience_Junior_Executive' and 'industry_Human Resources Management/Consulting'.

> Both LogisticRegressionCV snd RandomForest models fit the data well and give a high prediction accuracy of 89.8% with good precision and recall scores for both classes.

### Qn2

In [80]:
# Factors that distinguish job category

In [81]:
# Qn: What are the factors that distinguish data scientists from other data positions

In [82]:
# Filtering variables

In [83]:
target = 'job_title'
predictors = [i for i in jobst3.columns if i != target]

In [84]:
predictors

['location', 'salary', 'industry', 'experience', 'company_size', 'skills']

In [85]:
#numerical predictors
num_predictors = ['skills']
cat_predictors = [i for i in predictors if i != 'skills']

In [86]:
#Filtering numerical variables
from sklearn.feature_selection import f_classif, chi2

num_sig = []
f,p = f_classif(jobst3[['skills']], jobst3[target])
if p <= 0.05:
    num_sig.append(i)
print num_sig

['skills']


In [87]:
# The numerical feature 'skills' is significantly related to the target variable salary

In [88]:
cat_predictors

['location', 'salary', 'industry', 'experience', 'company_size']

In [89]:
# Get dataframe of categorical variables
jobst3_cat = jobst3.loc[:,cat_predictors]

In [90]:
jobst3_cat.head(2)

,location,salary,industry,experience,company_size
Index,,,,,
0,Singapore,1,Human Resources Management/Consulting,Junior_Executive,1
1,Singapore,0,Human Resources Management/Consulting,Junior_Executive,0


In [91]:
# Filtering the categorical variables
jobst3_cat = pd.get_dummies(jobst3_cat, drop_first=True)

In [92]:
#Convert the columns to type 'int64'
for i in jobst3_cat.columns:
    jobst3_cat[i] = jobst3_cat[i].astype('int64')

In [93]:
cat_sig = []
for i in jobst3_cat.columns:
    y = np.array(jobst3[target])
    ch, p = chi2(jobst3_cat[[i]],y)
    if p<=0.05:
        cat_sig.append(i)
print cat_sig        

['salary', 'company_size', 'location_Malaysia', 'location_Philippines', 'location_Singapore', 'industry_Call Center/IT-Enabled Services/BPO', 'industry_Chemical/Fertilizers/Pesticides', 'industry_Computer/Information Technology (Software)', 'industry_Education', 'industry_Electrical & Electronics', 'industry_R&D', 'industry_Science & Technology', 'industry_Wood/Fibre/Paper', 'experience_Less_than_1_year', 'experience_Manager', 'experience_Senior_Executive', 'experience_Staff']


In [94]:
#Building the filtered dataframe containing the significant variables
cat_sig_df = jobst3_cat.loc[:, cat_sig]
num_sig_df = jobst3.loc[:, num_sig]
jobst_sig = cat_sig_df.merge(num_sig_df, left_index=True, right_index=True, how='outer')
jobst_sig['job_title'] = jobst3['job_title']

In [95]:
jobst_sig.head(2)

,salary,company_size,location_Malaysia,location_Philippines,location_Singapore,industry_Call Center/IT-Enabled Services/BPO,industry_Chemical/Fertilizers/Pesticides,industry_Computer/Information Technology (Software),industry_Education,industry_Electrical & Electronics,industry_R&D,industry_Science & Technology,industry_Wood/Fibre/Paper,experience_Less_than_1_year,experience_Manager,experience_Senior_Executive,experience_Staff,skills,job_title
Index,,,,,,,,,,,,,,,,,,,
0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,17.0,Data Analyst
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,6.0,Data Analyst


### Train-test split plus Modeling

In [96]:
# Looking at the distribution of target variable
dist = jobst_sig['job_title'].value_counts()
dist

Business Analyst    374
Data Analyst        176
Data Scientist       68
Data Engineer        67
Name: job_title, dtype: int64

In [97]:
# Encode job_titles such that Business Analyst is 1 while other Data positions are 0
jobst_sig['job_title'] = jobst_sig['job_title'].apply(lambda x: 1 if x=='Data Scientist' else 0)

In [98]:
dist = jobst_sig['job_title'].value_counts()
dist

0    617
1     68
Name: job_title, dtype: int64

In [99]:
# Baseline accuracy
ratio_0 = float(dist[0])/(dist[0] + dist[1])
ratio_1 = float(dist[1])/(dist[0] + dist[1])
print 'class 0 ratio', ratio_0
print 'class 1 ratio', ratio_1
print 'Baseline accuracy', ratio_0

class 0 ratio 0.900729927007
class 1 ratio 0.0992700729927
Baseline accuracy 0.900729927007


In [100]:
from sklearn.model_selection import train_test_split

X = jobst_sig.loc[:, [i for i in jobst_sig.columns if i != 'job_title']]
y = jobst_sig.loc[:,'job_title']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, stratify=y, random_state=12)

In [101]:
#count of each class in y_train
num_zeros = (y_train == 0).sum()
num_ones = (y_train == 1).sum()
print num_zeros
print num_ones

431
48


In [102]:
# Oversampling
from imblearn.combine import SMOTEENN
sm = SMOTEENN(ratio={0:600,1:560}, random_state=12, n_jobs=-1)
X_sm, y_sm = sm.fit_sample(X_train, y_train)

/Users/dakshi/anaconda2/lib/python2.7/site-packages/imblearn/utils/deprecation.py:50: DeprecationWarning: 'k' is deprecated from 0.2 and will be removed in 0.4. Use 'k_neighbors' instead.
  category=DeprecationWarning)
/Users/dakshi/anaconda2/lib/python2.7/site-packages/imblearn/utils/deprecation.py:50: DeprecationWarning: 'm' is deprecated from 0.2 and will be removed in 0.4. Use 'm_neighbors' instead.
  category=DeprecationWarning)


In [103]:
#count of each class in y_train
num_zeros = (y_sm == 0).sum()
num_ones = (y_sm == 1).sum()
print num_zeros
print num_ones

427
440


In [104]:
# Modeling

In [105]:
# Fit first model with LogisticRegressionCV, second model with RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier

log = LogisticRegressionCV(penalty='l1', solver='liblinear', cv=3, n_jobs=-1)
log.fit(X_sm, y_sm)
y_pred = log.predict(X_test)
print 'score:', log.score(X_sm, y_sm)
print 'score:', log.score(X_test, y_test)

score: 0.863898500577
score: 0.742718446602


In [106]:
log_coefs = pd.DataFrame({'variable':X_train.columns, 'coef':log.coef_[0],'abs_coef':np.abs(log.coef_[0])})

log_coefs.sort_values('abs_coef', inplace=True, ascending=False)

# Factors that distinguish job category in order of decreasing impact
log_coefs

,abs_coef,coef,variable
10,24.346345,24.346345,industry_R&D
11,13.082959,13.082959,industry_Science & Technology
6,11.895424,11.895424,industry_Chemical/Fertilizers/Pesticides
5,6.876936,-6.876936,industry_Call Center/IT-Enabled Services/BPO
13,5.811725,-5.811725,experience_Less_than_1_year
3,5.583049,5.583049,location_Philippines
16,4.681830,4.681830,experience_Staff
9,4.213130,-4.213130,industry_Electrical & Electronics
4,3.527103,3.527103,location_Singapore
14,2.122743,2.122743,experience_Manager


In [107]:
print classification_report(y_test, y_pred)

             precision    recall  f1-score   support

          0       0.97      0.74      0.84       186
          1       0.24      0.75      0.36        20

avg / total       0.89      0.74      0.79       206



In [108]:
# RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100, random_state=12)
forest.fit(X_sm, y_sm)
y_pred = forest.predict(X_test)
print 'score:', forest.score(X_sm, y_sm)
print 'score:', forest.score(X_test, y_test)

score: 1.0
score: 0.78640776699


In [109]:
forest_features = pd.DataFrame({'variable':X_train.columns, 'coef':forest.feature_importances_})

forest_features.sort_values('coef', inplace=True, ascending=False)

# Factors that distinguish job category in order of decreasing impact
forest_features

,coef,variable
1,0.256270,company_size
17,0.144605,skills
0,0.110773,salary
14,0.086253,experience_Manager
10,0.068906,industry_R&D
3,0.059021,location_Philippines
7,0.054132,industry_Computer/Information Technology (Soft...
15,0.044823,experience_Senior_Executive
4,0.043451,location_Singapore
5,0.037967,industry_Call Center/IT-Enabled Services/BPO


In [110]:
print classification_report(y_test, y_pred)

             precision    recall  f1-score   support

          0       0.94      0.81      0.87       186
          1       0.24      0.55      0.33        20

avg / total       0.88      0.79      0.82       206

